<a href="https://colab.research.google.com/github/justtuananh/Learn_Huggies_face/blob/main/train_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers 
!pip install datasets

In [10]:
from datasets import load_dataset 
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example): 
    return tokenizer(example["sentence1"], example["sentence2"], truncation = True )

tokenized_datasets = raw_datasets.map(tokenize_function, batched = True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# chuẩn bị cho huấn luyện 
# hậu xử lí cho dataset của mình 

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

ĐỊNH NGHĨA LẠI BỘ DỮ LIỆU CỦA MÌNH

In [11]:
from torch.utils.data import DataLoader 

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle = True, batch_size = 8, collate_fn = data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size = 8, collate_fn = data_collator
)

KIỂM TRA DỮ LIỆU CỦA MỘT LÔ XEM NTN 


In [15]:
for batch in train_dataloader: 
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 77]),
 'token_type_ids': torch.Size([8, 77]),
 'attention_mask': torch.Size([8, 77])}

KHỞI TẠO MÔ HÌNH NHƯ Ở LẦN TRƯỚC 

In [16]:
from transformers import AutoModelForSequenceClassification 

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels = 2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.7016, grad_fn=<NllLossBackward0>) torch.Size([8, 2])
